# 노래방 New Title List

## 필요 라이브러리 로드

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import os, math

In [10]:
import platform
from matplotlib import font_manager, rc

# 한글 폰트 지정
path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system()=="Darwin":
    rc('font', family='AppleGothic')
else:
    print("Unknown System")

In [11]:
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False # 그래프에서 음수 표현 가능하도록 설정.

## 파일 불러오기

In [31]:
f_path = 'c:/data/Wantreez/Crawling/music'
try:
    os.chdir(f_path)
except:
    pass

## 날짜 설정

In [32]:
# 전 주 파일 날짜(일주일)
start_date = '2022-05-02'
end_date = '2022-05-08'
# 비교할 파일 날짜(일주일)
new_start_date = '2022-05-09'
new_end_date = '2022-05-15'
new_dates_df = pd.date_range(new_start_date, new_end_date).astype(str)

In [33]:
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/music'

## 1주일 동안 새로 진입한 곡 추출

In [34]:
site_list = ['ky', 'tj']
for site_name in site_list:
    # 실시간 순위 파일 폴더로 이동
    live_folder = f'/crawled_data/live_{site_name}/'
    try:
        os.chdir(code_path + live_folder)
    except:
        pass
    file_list = os.listdir()

    # 수집한 파일 모두 합치기
    total_df = pd.DataFrame()
    for one in file_list:
        temp = pd.read_excel(one)
        total_df = pd.concat([total_df, temp], ignore_index=True)
    total_df['날짜'] = total_df['날짜'].astype(str)

    before_df = total_df[ (total_df['날짜'] >= start_date) & (total_df['날짜'] <= end_date)].reset_index(drop=True)
    new_df = total_df[ (total_df['날짜'] >= new_start_date) & (total_df['날짜'] <= new_end_date)].reset_index(drop=True)
    # 전주 노래 목록
    title_list = before_df['곡'].unique()
    # 이번주 새로 진입한 곡 추출
    pop_title_list = []
    pop_artist_list = []
    pop_date_df = pd.DataFrame()
    for new_one_date in new_dates_df:
        new_date_df = total_df[total_df['날짜'] == new_one_date].reset_index(drop=True)

        for i in range(new_date_df.shape[0]):
            one_title = new_date_df['곡'][i]
            one_artist = new_date_df['가수'][i]
            if one_title not in title_list:
                pop_title_list.append(one_title)
                pop_artist_list.append(one_artist)

        for two_title, two_artist in zip(pop_title_list, pop_artist_list):
            temp = new_df[ (new_df['곡'] == two_title) & (new_df['가수'] == two_artist)]
            pop_date_df = pd.concat([pop_date_df, temp], ignore_index=True)
            pop_date_df = pop_date_df.drop_duplicates(['날짜', '곡', '가수'], ignore_index=True)
    
    pop_df = pop_date_df.drop_duplicates(['곡', '가수']).reset_index(drop=True)
    pop_df = pop_df[['날짜', '순위', '곡', '가수', '사이트']]

     # 파일 저장
    save_folder = f'/crawled_data/outlier_data/{new_start_date}_{new_end_date}/{site_name}/'
    path = code_path + save_folder
    os.makedirs(path, exist_ok=True)
    file_name = f'[{site_name}]{new_start_date}_{new_end_date}_pop_df.xlsx'
    pop_df.to_excel(path + file_name, index=False, encoding='utf-8')
    print(f'{file_name} 출력완료.')

[ky]2022-05-09_2022-05-15_pop_df.xlsx 출력완료.
[tj]2022-05-09_2022-05-15_pop_df.xlsx 출력완료.


# 코드 마지막